In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

In [ ]:
import plot
import util

### Load Data ###

In [ ]:
mri_features = xr.open_dataset("../data/processed/mri-features-all.nc").load()
mri_factors = xr.open_dataset("../data/processed/mri-features-all-fa.nc").load()

In [ ]:
def read_cibersort_results(fn):
    df = pd.read_table(fn)
    df = df.rename(columns={
        'Input Sample': 'case',
    })
    df = df.set_index('case')
    df_cf = df.copy()
    del df_cf['P-value']
    del df_cf['Pearson Correlation']
    del df_cf['RMSE']
    da_cf = xr.DataArray(df_cf, dims=['case', 'cell_type'])
    ds = xr.Dataset({
        'cell_fraction': da_cf,
        'p': df['P-value'],
        'correlation': df['Pearson Correlation'],
        'rmse': df['RMSE'],
    })
    
    return ds
    
cibersort = read_cibersort_results("../models/cibersort/CIBERSORT.Output.txt")
cibersort = cibersort.reindex_like(mri_features)

In [ ]:
numeric_mri_features = list(set(mri_features.keys()) - {'case', 'Comment', 'MultiFocal'})
mri_features_da = mri_features[numeric_mri_features].to_array('cad_feature')

In [ ]:
cf_m_cor = util.cor(cibersort['cell_fraction'], mri_features_da, 'case')
cf_m_cor['p'] = np.fmin(1, cf_m_cor['nominal_p'] * cf_m_cor['nominal_p'].size)

In [ ]:
plot.heatmap(cf_m_cor['correlation'], cmap='coolwarm', row_dendrogram=True)

In [ ]:
plot.heatmap(cf_m_cor['correlation'], mask=cf_m_cor['p'] > 0.05, cmap='coolwarm', row_dendrogram=True)

In [ ]:
list(cf_m_cor['cad_feature'][(cf_m_cor['p'] < 0.05).sum('cell_type') > 0].values)

In [ ]:
cf_f_cor = util.cor(cibersort['cell_fraction'].reindex_like(mri_factors), mri_factors['factors'], 'case')
cf_f_cor['p'] = np.fmin(1, cf_f_cor['nominal_p'] * cf_f_cor['nominal_p'].size)

In [ ]:
plot.heatmap(cf_f_cor['correlation'], cmap='coolwarm', row_dendrogram=True)

In [ ]:
plot.heatmap(cf_f_cor['correlation'], mask=cf_f_cor['p'] > 0.05, cmap='coolwarm', row_dendrogram=True)